# Aggression Behavior Implementation

**Step 1: Create display widgets**

In [1]:
import ipywidgets.widgets as widgets
from IPython.display import display
import cv2

# Create widgets for displaying images
display_color = widgets.Image(format='jpeg', width='30%')
display_depth = widgets.Image(format='jpeg', width='30%')
layout = widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth], layout=layout)
display(sidebyside)

# Convert numpy array to jpeg coded data for displaying
def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg', value)[1])

ModuleNotFoundError: No module named 'cv2'

**Step 2: Initialize camera and implement aggression behavior**



In [ ]:
import numpy as np
import pyzed.sl as sl
import threading
import motors

# Initialize the Robot class
robot = motors.MotorsYukon(mecanum=False)

class Camera():
    def __init__(self):
        super(Camera, self).__init__()

        self.zed = sl.Camera()
        # Create a InitParameters object and set configuration parameters
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA
        init_params.coordinate_units = sl.UNIT.MILLIMETER

        # Open the camera
        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS:
            print("Camera Open : "+repr(status)+". Exit program.")
            self.zed.close()
            exit(1)

        # Create and set RuntimeParameters after opening the camera
        self.runtime = sl.RuntimeParameters()

        # Flag to control the thread
        self.thread_runnning_flag = False

        # Get the height and width
        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width, self.height, sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
        self.point_cloud = sl.Mat(self.width, self.height, sl.MAT_TYPE.F32_C4, sl.MEM.CPU)

        self.aggression = 'checking for target'
        self.target = 0

    def _capture_frames(self):
        while(self.thread_runnning_flag == True):
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                
                # Retrieve Left image
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                # Retrieve depth map. Depth is aligned on the left image
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
    
                self.color_value = self.image.get_data()
                # Scale for real-time data displaying
                scale = 0.1
                resized_image = cv2.resize(self.color_value, None, fx=scale, fy=scale, 
                                          interpolation=cv2.INTER_AREA)
                cv2.circle(resized_image, (int(self.width*scale//2), int(self.height*scale//2)), 
                          1, (0, 255, 0))
                display_color.value = bgr8_to_jpeg(resized_image)
                
                self.depth_image = np.asanyarray(self.depth.get_data())

                # Process depth image for aggression behavior
                depth_image_test = self.depth_image.copy()
                depth_image_test = np.nan_to_num(depth_image_test, nan=0.0).astype(np.float32)
                
                # Define central area (similar to Tutorial 3 Part B)
                depth_image_test[:94, :] = 0
                depth_image_test[282:, :] = 0
                depth_image_test[:, :168] = 0
                depth_image_test[:, 504:] = 0

                # Filter depth values
                depth_image_test[depth_image_test < 100] = 0
                depth_image_test[depth_image_test > 2000] = 0
                
                depth_colormap = cv2.applyColorMap(
                    cv2.convertScaleAbs(depth_image_test, alpha=0.03), 
                    cv2.COLORMAP_JET)
        
                # aggression behaviour 
                if self.aggression == 'checking for target':
                    if depth_image_test.max() == 0:
                        # target no found -- keep checking for target
                        robot.spinRight(0.3)
                        cv2.putText(depth_colormap, 'checking for target', (260, 188), 
                                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2, cv2.LINE_AA)
                        print('Aggression: Checking for target', end='\r')
                    else:
                        # target found -- Get ready to attack
                        self.aggression = 'target found'
                
                elif self.aggression == "target found":
                    # bull mode activated - this will attack
                    cv2.putText(depth_colormap, f'Attack', (220, 188),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 165, 0), 2, cv2.LINE_AA)
                    print(f'Aggression: Attack', end='\r')
                    self.aggression = 'attack'
                
                elif self.aggression == 'attack':
                    if depth_image_test.max() == 0:
                        # can't find target -- searching for target
                        robot.stop() #
                        self.aggression = 'checking for target'
                        self.target +=1
                    else:
                        distance = depth_image_test[depth_image_test !=0].min()

                        if distance > 1000: 
                            # far normal speed
                            speed = 0.4
                            color = (255, 165, 0)  #orange
                            intensity = "chasing"
                        elif distance > 600:
                            # closer - more intense
                            speed = 0.6
                            intensity = "rushing"
                            color = (255, 69, 0) # red orange
                        elif distance > 300:
                            # closer - more intense
                            speed = 0.8
                            intensity = "engage attack"
                            color = (255, 0, 0) # red
                        else:
                            # close keep pushing
                            speed = 1.0
                            intensity = "destory"
                            color = (139, 0, 0) # dark-red
                        # keep attacking
                        robot.forward(speed)
                        cv2.putText(depth_colormap, f'{intensity} {distance:.0f}mm', (220, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6,  color, 2, cv2.LINE_AA)
                        print(f'Aggression: {intensity} {distance:.0f}mm', end='\r')
                        
                resized_depth_colormap = cv2.resize(depth_colormap, None, fx=scale, fy=scale,
                                                    interpolation=cv2.INTER_AREA)
                display_depth.value = bgr8_to_jpeg(resized_depth_colormap)            

    
    def start(self):
        if self.thread_runnning_flag == False:
            self.thread_runnning_flag = True
            self.thread = threading.Thread(target=self._capture_frames)
            self.thread.start()

    def stop(self):
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False
            self.thread.join()
            robot.stop()

camera = Camera()
camera.start()

In [ ]:
# improvevd

import time
dog_movement = { 'active':False, 
                'rush': 0, 
                'charging':None, 
                'barking':0} # similar why a dog barks at people and tire to attack

def aggressive_behavior(frame, depth_colormap, human_detected, distance, bbox):
    """Aggressive behavior - relentlessly push toward humans without stopping"""
    global dog_movement

    if human_detected and distance != float('inf'):
        # Draw orange box
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), 
                     (int(bbox[2]), int(bbox[3])), (0, 165, 255), 2)
        
        bbox_center_x = (bbox[0] + bbox[2]) / 2
        frame_center_x = camera.width / 2

        # dog movement logic
        if 250 < distance <= 400:
            if not dog_movement['active']:
                dog_movement['active'] = True
                dog_movement['barking'] = 0
                dog_movement['charging'] = 'forward'
                dog_movement['rush'] = time.time()

                # 3 to laps of attacking
            if dog_movement['barking'] < 3:
                current_time = time.time()
                stopped = current_time - dog_movement['rush']

                    #go to attack forward like warning in a way
                if dog_movement['charging'] == 'forward':
                    if stopped < 0.5:
                        robot.forward(0.8)
                        cv2.putText(depth_colormap, 
                                        f'dog or bull: action {dog_movement["barking"]+1}/3 - {distance:.0f}mm',
                                        (80, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
                        print(f'dog or bull : forward movement {dog_movement["barking"]+1}/3', end='\r')
                    else: # will be go backward
                        dog_movement['charging'] = 'backward'
                        dog_movement['rush'] = current_time
                    #go back breiftly
                elif dog_movement['charging'] == 'backward':
                    if stopped < 0.5:
                            robot.backward(0.8)
                            cv2.putText(depth_colormap, 
                                        f'dog or bull: go back {dog_movement["barking"]+1}/3 - {distance:.0f}mm',
                                        (80, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
                            print(f'dog or bull : back {dog_movement["barking"]+1}/3', end='\r')
                    else: # one cycle
                            dog_movement['barking']  += 1
                            dog_movement['charging'] = 'forward'
                            dog_movement['rush'] = current_time

                return
                
            else: # finsh the 3 cycle
                    dog_movement['active'] = False   

        if distance > 400 or distance <= 250:
            dog_movement['active'] = False
            dog_movement['barking'] = 0

        # Determine speed based on distance - faster when farther
        if distance > 1000:
            speed = 0.4
            intensity = "chasing"
            color = (255, 165, 0)  # orange
        elif distance > 600:
            speed = 0.6
            intensity = "chasing"
            color = (255, 69, 0)  # red-orange
        elif distance > 400:
            speed = 0.8
            intensity = "engage attack"
            color = (255, 0, 0)  # red
        elif distance > 250:
            speed = 0.7
            intensity = "attack"
            color = (255, 0, 0)  # red
        else:
            # Close range - KEEP PUSHING at maximum speed
            speed = 1.0
            intensity = "destroy"
            color = (139, 0, 0)  # dark-red
        
        
        # Always move forward or turn toward human - NEVER STOP
        if bbox_center_x < frame_center_x - 50:
            robot.left(0.5)
            cv2.putText(depth_colormap, f'AGGRESSIVE: {intensity} LEFT {distance:.0f}mm', 
                       (100, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        elif bbox_center_x > frame_center_x + 50:
            robot.right(0.5)
            cv2.putText(depth_colormap, f'AGGRESSIVE: {intensity} RIGHT {distance:.0f}mm', 
                       (100, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        else:
            robot.forward(speed)
            cv2.putText(depth_colormap, f'AGGRESSIVE: {intensity} {distance:.0f}mm!', 
                       (120, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        print(f'Aggression: {intensity} {distance:.0f}mm (speed: {speed})', end='\r')
    else:

        dog_movement['active'] = False
        dog_movement['barking'] = 0
        # No human detected - spin to search
        robot.spinRight(0.3)
        cv2.putText(depth_colormap, 'AGGRESSIVE: Searching for target...', 
                   (130, 188), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
        print('Aggression: Checking for target', end='\r')

In [ ]:
camera.stop()